In [2]:
import pandas as pd

In [61]:
tabela = pd.read_csv('vozila brez nepotrebnih vejc in euro.csv', sep=';', encoding='latin1', low_memory=False)
tabela = tabela[['Registrsko obmocje', 'Masa', 'Znamka', 'Masa', 'ccm', 'Moc', 'Gorivo', 'Najvisja hitrost', 'CO2', 'Poraba goriva', 'Okoljevarstvena oznaka', 'Leto izdelave']]

In [62]:
tabela[:10]

,Registrsko obmocje,Masa,Znamka,Masa,ccm,Moc,Gorivo,Najvisja hitrost,CO2,Poraba goriva,Okoljevarstvena oznaka,Leto izdelave
0,CELJE,2485,BENTLEY,2485,5998,412.0,P,323,410.0,17.1,4,2005
1,CELJE,2085,MERCEDES BENZ,2085,6209,386.0,P,300,335.0,14.9,4,2008
2,CELJE,1830,BMW,1830,4999,373.0,P,250,357.0,14.8,0,2005
3,CELJE,1435,PORSCHE,1435,3996,368.0,P,310,326.0,13.8,5,2012
4,CELJE,2245,PORSCHE,2245,4806,368.0,P,278,270.0,11.5,5,2010
5,CELJE,2045,PORSCHE,2045,4806,368.0,P,303,286.0,12.2,5,2010
6,CELJE,1210,PORSCHE,1210,3600,353.0,P,310,326.0,13.6,4,2007
7,CELJE,1695,PORSCHE,1695,3600,353.0,P,310,326.0,13.6,4,2007
8,CELJE,2070,MERCEDES BENZ,2070,4663,335.0,P,250,225.0,9.6,6,2015
9,CELJE,1635,BMW,1635,2979,331.0,P,250,194.0,8.3,6,2016


In [53]:
poraba = tabela[['Registrsko obmocje', 'Poraba goriva']]

In [54]:
skupna_po_krajih = poraba.groupby('Registrsko obmocje')['Poraba goriva'].mean()
poraba_po_krajih = pd.DataFrame(skupna_po_krajih).sort_values('Poraba goriva')
PORABA = poraba_po_krajih
PORABA

,Poraba goriva
Registrsko obmocje,
NOVO MESTO,5.708461
KOPER,5.800160
POSTOJNA,5.855409
KRSKO,5.857492
MARIBOR,5.862853
KRANJ,5.876111
MURSKA SOBOTA,5.913631
SLOVENJ GRADEC,5.928654
NOVA GORICA,5.941255


In [55]:
moc = tabela[['Registrsko obmocje', 'Moc']]

In [56]:
moc_kraji = moc.groupby('Registrsko obmocje')['Moc'].mean()
moc_po_krajih = pd.DataFrame(moc_kraji).sort_values('Moc')
MOC = moc_po_krajih
MOC

,Moc
Registrsko obmocje,
MURSKA SOBOTA,78.441921
NOVO MESTO,79.755520
KRSKO,81.116119
MARIBOR,81.148100
KRANJ,81.494087
CELJE,81.639676
NOVA GORICA,81.844493
SLOVENJ GRADEC,82.887269
POSTOJNA,83.209338


In [57]:
co2 = tabela[['Registrsko obmocje', 'CO2']]

In [58]:
co2_kraji = co2.groupby('Registrsko obmocje')['CO2'].mean()
co2_po_krajih = pd.DataFrame(co2_kraji).sort_values('CO2')
CO2 = co2_po_krajih
CO2

,CO2
Registrsko obmocje,
NOVO MESTO,142.099415
LJUBLJANA,144.079255
MURSKA SOBOTA,144.198351
CELJE,144.417368
MARIBOR,144.462349
KRSKO,144.919896
POSTOJNA,145.467107
KRANJ,145.626757
KOPER,145.684547


In [59]:
leto = tabela[['Registrsko obmocje', 'Leto izdelave']]

In [60]:
leto_kraji = leto.groupby('Registrsko obmocje')['Leto izdelave'].mean()
leto_po_krajih = pd.DataFrame(leto_kraji).sort_values('Leto izdelave')
LETO = leto_po_krajih
LETO

,Leto izdelave
Registrsko obmocje,
NOVA GORICA,2008.553763
KRSKO,2008.753459
KOPER,2009.070819
NOVO MESTO,2009.086404
CELJE,2009.097007
KRANJ,2009.226889
MARIBOR,2009.301816
SLOVENJ GRADEC,2009.347841
MURSKA SOBOTA,2009.451257


In [67]:
euro = tabela[['Registrsko obmocje', 'Okoljevarstvena oznaka']]

In [66]:
euro_kraji = euro.groupby('Registrsko obmocje')['Okoljevarstvena oznaka'].mean()
euro_po_krajih = pd.DataFrame(euro_kraji).sort_values('Okoljevarstvena oznaka')
EURO = euro_po_krajih
EURO

,Okoljevarstvena oznaka
Registrsko obmocje,
NOVA GORICA,4.218413
KRSKO,4.269966
NOVO MESTO,4.323958
KOPER,4.334955
CELJE,4.337543
KRANJ,4.357914
MARIBOR,4.370308
SLOVENJ GRADEC,4.387249
MURSKA SOBOTA,4.411332


In [68]:
#HURA, leto izdelave se ujema z okoljevarstveno oznako!!

In [71]:
pd.merge(EURO, LETO, on='')

KeyError: ''